In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [7]:
# /// make sure you save changes in the excel sheet before running the script ///
import pandas as pd

require_cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# only read specific columns from an excel file


#-----------------------------------PUT FILE PATHS IN HERE---------------------------------------------
#--------------------------------------------------------------------------------------------------


# FILE INPUT LOCATION -- "USE THIS FILE"
file_to_use = r'\\bos-700-11\users$\DAdams\Desktop\MyStuff\Excel_UBS_Parse_Temp.csv'
ubs_sheet = pd.read_csv(file_to_use)


# FILE OUTPUT DESTINATION FOLDER
ouput_place = r'\\bos-700-11\users$\DAdams\Desktop\MyStuff\UBSParseOutput'

#--------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------



#formats UBS sheet to DTC numbers
ubs_sheet = ubs_sheet.astype({'ExecutingBrokerDTCCode':'string'})
    
ubs_sheet['ExecutingBrokerDTCCode'] = "000" + ubs_sheet['ExecutingBrokerDTCCode']
    
ubs_sheet['ExecutingBrokerDTCCode'] = ubs_sheet['ExecutingBrokerDTCCode'].str[-4:]
    


    
    


#-----------------------------------PUT ERRORS IN HERE---------------------------------------------
#--------------------------------------------------------------------------------------------------

# List of 'bad' accounts/cusips that need to be removed from the sheet
find_aorc_to_remove = ["64990G7W6"]

#--------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------


    
# used as a list of indexes to remove / work with
dropperlist = []


# creates a list of indexes with errored accounts/cusips
def findListOfBaddies(ubs_dataframe, baddies):
    
    for baddy in baddies:
        if len(baddy) != 9:
            findBaddy(baddy, ubs_dataframe, 'AccountName')
        else:
            findBaddy(baddy, ubs_dataframe, 'SecurityCUSIP')
        
        
# individual funx ran for each item in baddies list
def findBaddy(removethis, mainsheet, colnum):
    
    #setting the column we are running this on
    colarray = mainsheet[colnum]
    
    # iterator to store the index that has the errors
    rowfinder = 0
    
    for aorc in colarray:
        if removethis == aorc:
            dropperlist.append(rowfinder)
            
        rowfinder = rowfinder + 1        




# REMOVES CERTAIN A LIST OF SPECIFIC ROWS FROM A DATAFRAME - takes in a df, bad accounts, and an empty list
# PUTS ALL REMOVED INDEXES INTO A NEW LIST via createErrorSheet

def removeBaddies (ubs_dataframe, baddies, droplist):
    
    findListOfBaddies(ubs_dataframe, baddies)
    
    
    #grabs a list of all the index's
    full_index_list = list(range(ubs_dataframe.index.start, ubs_dataframe.index.stop))
    
    #filter out drop list from full list to get filtered drop list
    filtered_droplist = [i for i in full_index_list if i not in droplist]
   
    #list of all good accounts remaining
    global good_sheet 
    
    good_sheet = ubs_dataframe.drop(ubs_dataframe.index[droplist])
    
    #list of only errored accounts
    global error_sheet
    
    error_sheet = ubs_dataframe.drop(ubs_dataframe.index[filtered_droplist])
    
    
def uploader():
    
    #ouput names
    error_sheet.to_csv(ouput_place + "\Excel_UBS_Test_bad.csv", index=False)
    good_sheet.to_csv(ouput_place + "\Excel_UBS_Test_good.csv", index=False)
    ubs_sheet.to_csv(ouput_place + "\Excel_UBS_Test_backup.csv", index=False)

   

 # RUNS FUNCTIONS -------------------------------------


#print(ubs_sheet)

removeBaddies(ubs_sheet, find_aorc_to_remove, dropperlist)
uploader()


